In [1]:
import sys

# extra steps might be required on Mac https://github.com/KimiNewt/pyshark/#mac-os-x
!{sys.executable} -m pip install -U numpy matplotlib pyshark nest_asyncio

Requirement already up-to-date: numpy in c:\users\felix\appdata\local\programs\python\python38\lib\site-packages (1.18.1)
Requirement already up-to-date: matplotlib in c:\users\felix\appdata\local\programs\python\python38\lib\site-packages (3.1.3)
Requirement already up-to-date: pyshark in c:\users\felix\appdata\local\programs\python\python38\lib\site-packages (0.4.2.9)
Requirement already up-to-date: nest_asyncio in c:\users\felix\appdata\local\programs\python\python38\lib\site-packages (1.2.3)


You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
import asyncio
import nest_asyncio

# patch asyncio to work in notebook to make pyshark work
# requires Python 3.8 if using Windows
loop = asyncio.DefaultEventLoopPolicy().new_event_loop()
nest_asyncio.apply(loop=loop)

In [3]:
import pyshark

commute_16122019 = pyshark.FileCapture('capture-0-a4.pcap', eventloop=loop)
commute_17122019 = pyshark.FileCapture('capture-1-a4.pcap', eventloop=loop)
commute_18122019 = pyshark.FileCapture('capture-2-a4.pcap', eventloop=loop)
commute_13012020 = pyshark.FileCapture('capture-5-a4.pcap', eventloop=loop)
commute_14012020 = pyshark.FileCapture('capture-6-a4.pcap', eventloop=loop)
commute_20012020 = pyshark.FileCapture('capture-7-a4.pcap', eventloop=loop)
